In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler #para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from sklearn import preprocessing

## Device config

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Esto garantiza que se ejecutara en GPU si esta disponible
print(device)

cpu


## MNIST Data

In [3]:
df_raw = pd.read_csv('D:/.Memoria/Test/Jupyter/test_example.csv')
columns = list(df_raw.columns)
#  Eliminar los con tiempos de ejecución 0 o muy altos
df_clean = shuffle(df_raw[(df_raw['ql_rt_msec'] > 0) & (df_raw['ql_rt_msec'] < 1e4)])
#df_clean = shuffle(df_raw)

df_clean.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean.csv',index=False)

### HAY QUE CONVERTIR ALGUNAS COLUMNAS Q ESTAN EN PORCENTAJE A FLOAT
df_clean['ql_rt_clocks'] = df_clean['ql_rt_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_seg'] = df_clean['ql_same_seg'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_page'] = df_clean['ql_same_page'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_cl_wait_clocks'] = df_clean['ql_cl_wait_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_clocks'] = df_clean['ql_c_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_cl_wait'] = df_clean['ql_c_cl_wait'].apply(lambda x: float(x.strip('%'))/100)


print(f'shape df_raw: {df_raw.shape}')

df_clean = df_clean[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait','ql_rt_msec']]
print(f'shape df_clean: {df_clean.shape}')


msk = np.random.rand(len(df_clean)) <= 0.8
df_train = df_clean[msk]
df_test = df_clean[~msk]

df_train.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_train.csv',index=False)
df_test.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_test.csv',index=False)

#print(f'shape df_train: {df_train.shape}')
#print(f'shape df_test: {df_test.shape}')


## FEATURES - TRAIN Y TEST
X_df_train = df_train[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait']]
X_df_test = df_test[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait']]

X_numpy_train = X_df_train.to_numpy().astype(np.float32)
X_numpy_test = X_df_test.to_numpy().astype(np.float32)

###SOLO PA COMPARAR NO SIRVEN DE NADA DESPUES BORRAR
X_numpy_train_old =  X_numpy_train
X_numpy_test_old = X_numpy_test

######

X_numpy_train = preprocessing.normalize(X_numpy_train)
X_numpy_test = preprocessing.normalize(X_numpy_test)

#X_numpy_train = preprocessing.scale(X)


## TARGETS - TRAIN Y TEST
y_df_train = df_train['ql_rt_msec']
y_df_test = df_test['ql_rt_msec']

y_numpy_train = y_df_train.to_numpy().astype(np.float32)
y_numpy_test = y_df_test.to_numpy().astype(np.float32)


# Pasarlos a Torch. 
X_train = torch.from_numpy(X_numpy_train)
X_test = torch.from_numpy(X_numpy_test)
y_train = torch.from_numpy(y_numpy_train)
y_test = torch.from_numpy(y_numpy_test)
# También pasar los targets de vector fila a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print("-----------------------")
print(f'shape X_numpy_train: {X_numpy_train.shape}')
print(f'shape X_numpy_test: {X_numpy_test.shape}')
print(f'shape y_numpy_train: {y_numpy_train.shape}')
print(f'shape y_numpy_test: {y_numpy_test.shape}')
print("-----------------------")
print(f'shape X_train: {X_train.shape}')
print(f'shape X_test: {X_test.shape}')
print(f'shape y_train: {y_train.shape}')
print(f'shape y_test: {y_test.shape}')
print("-----------------------")

shape df_raw: (3711, 21)
shape df_clean: (2253, 15)
-----------------------
shape X_numpy_train: (1805, 14)
shape X_numpy_test: (448, 14)
shape y_numpy_train: (1805,)
shape y_numpy_test: (448,)
-----------------------
shape X_train: torch.Size([1805, 14])
shape X_test: torch.Size([448, 14])
shape y_train: torch.Size([1805, 1])
shape y_test: torch.Size([448, 1])
-----------------------


In [4]:
print(X_numpy_train_old)

[[0.0000e+00 9.4000e-01 2.8717e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0000e+01 6.9000e-01 3.0141e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 4.6000e-01 3.8700e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [0.0000e+00 6.9000e-01 8.5400e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 6.6000e-01 9.0500e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 5.3000e-01 1.3600e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]]


In [5]:
print(X_numpy_train)

[[0.0000000e+00 2.9594916e-05 9.0412468e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.0425008e-04 1.3946628e-05 6.0922509e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 8.4397070e-05 7.1003622e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 2.1210554e-04 2.6251903e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 7.4172793e-05 1.0170663e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 2.1920496e-04 5.6248821e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


## Parametros

In [6]:
n_samples_train, n_features_train = X_train.shape
n_samples_test, n_features_test = X_test.shape
learning_rate = 0.01
hidden_size = 10000
input_size = n_features_train 

## Modelo


In [7]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, 1)
        
    # Se aplica el modelo construido arriba en el forward    
    def forward(self,x):
        x=self.l1(x)
        x=F.relu(x)
        x=self.l2(x)
        return x
        
# Definimos el modelo con la clase
model = NeuralNet(input_size,hidden_size).to(device)

## Loss y optimizer

In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

## Training Loop

In [9]:
for epoch in range(100000):
    y_hat = model(X_train)
    loss = criterion(y_hat,y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1)%500==0:
        print(f'epoch: {epoch+1}, loss = {loss.item()}')

epoch: 500, loss = 531971.0625
epoch: 1000, loss = 509506.40625
epoch: 1500, loss = 493023.25
epoch: 2000, loss = 479328.46875
epoch: 2500, loss = 466792.0
epoch: 3000, loss = 455479.25
epoch: 3500, loss = 445508.625
epoch: 4000, loss = 437506.53125
epoch: 4500, loss = 430225.4375
epoch: 5000, loss = 423965.71875
epoch: 5500, loss = 418345.4375
epoch: 6000, loss = 414298.96875
epoch: 6500, loss = 409855.90625
epoch: 7000, loss = 406538.15625
epoch: 7500, loss = 402214.4375
epoch: 8000, loss = 399180.75
epoch: 8500, loss = 395798.96875
epoch: 9000, loss = 392430.40625
epoch: 9500, loss = 390519.8125
epoch: 10000, loss = 387629.625
epoch: 10500, loss = 384515.0
epoch: 11000, loss = 382565.90625
epoch: 11500, loss = 378975.09375
epoch: 12000, loss = 376774.9375
epoch: 12500, loss = 374723.5
epoch: 13000, loss = 373153.5
epoch: 13500, loss = 371407.0
epoch: 14000, loss = 369442.3125
epoch: 14500, loss = 366950.8125
epoch: 15000, loss = 365412.0
epoch: 15500, loss = 362755.78125
epoch: 1600

In [10]:
with torch.no_grad():
    print(X_test.shape)
    print(X_test[0])

torch.Size([448, 14])
tensor([9.6300e-01, 4.0928e-05, 2.4075e-01, 1.2110e-01, 1.1917e-04, 9.6252e-07,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 2.4075e-04, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])


In [11]:


with torch.no_grad():
    outputs = model(X_test)
    for i in range(0,outputs.shape[0]):
        print(y_test[i],outputs[i])

tensor([1.]) tensor([15.6625])
tensor([1.]) tensor([16.3378])
tensor([2.]) tensor([-20.5837])
tensor([1.]) tensor([-20.1050])
tensor([1.]) tensor([41.7896])
tensor([1.]) tensor([-63.6677])
tensor([1.]) tensor([-1501.9828])
tensor([370.]) tensor([3192.5928])
tensor([1.]) tensor([10.7945])
tensor([24.]) tensor([106.6537])
tensor([16.]) tensor([-223.8991])
tensor([82.]) tensor([332.0003])
tensor([2.]) tensor([42.1116])
tensor([87.]) tensor([52.5320])
tensor([2.]) tensor([-32.9929])
tensor([20.]) tensor([2.9611])
tensor([1.]) tensor([16.1994])
tensor([22.]) tensor([59.5294])
tensor([776.]) tensor([-1023.7979])
tensor([57.]) tensor([1470.5153])
tensor([17.]) tensor([374.5022])
tensor([1.]) tensor([-65.7557])
tensor([3.]) tensor([-4.2853])
tensor([2.]) tensor([-237.0142])
tensor([200.]) tensor([546.8312])
tensor([12.]) tensor([133.6403])
tensor([51.]) tensor([492.7062])
tensor([1.]) tensor([2.1632])
tensor([1.]) tensor([-22.3209])
tensor([1.]) tensor([-27.2620])
tensor([8.]) tensor([3.5528])

tensor([1.]) tensor([45.8637])
tensor([1153.]) tensor([876.4673])
tensor([36.]) tensor([101.9665])
tensor([1.]) tensor([-20.7757])
tensor([1.]) tensor([-6.8080])
tensor([3.]) tensor([57.5037])
tensor([3.]) tensor([-34.7802])
tensor([75.]) tensor([171.0341])
tensor([2.]) tensor([31.7254])
tensor([4.]) tensor([295.1380])
tensor([1.]) tensor([-1.9683])
tensor([3.]) tensor([-4.5226])
tensor([5.]) tensor([6.0291])
tensor([415.]) tensor([367.4075])
tensor([6.]) tensor([-241.9024])
tensor([1.]) tensor([-1083.0909])
tensor([1.]) tensor([-65.7557])
tensor([19.]) tensor([-18.3977])
tensor([6.]) tensor([109.7492])
tensor([1.]) tensor([-22.4702])
tensor([5.]) tensor([59.4067])
tensor([571.]) tensor([2911.5610])
tensor([3.]) tensor([8.4643])
tensor([2.]) tensor([-16.7671])
tensor([17.]) tensor([-88.9515])
tensor([1.]) tensor([1531.3873])
tensor([2.]) tensor([-1.8566])
tensor([100.]) tensor([119.8838])
tensor([1.]) tensor([-53.0516])
tensor([4.]) tensor([26.2931])
tensor([2.]) tensor([7.4293])
tenso

In [ ]:
# SIEMPRE EVITAR CONSIDERAR LOS GRADIENTES
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # Debemos hacer nuevamente el reshape OJO
        images = images.reshape(-1, 28*28).to(device) #Se envia el GPU si esta disponible...
        labels = labels.to(device)
        outputs = model(images)
        
        # value, index
        _, predictions = torch.max(outputs, 1) # obtener el max posible valor
        n_samples += labels.shape[0] # numero de muestras por lote actual
        n_correct += (predictions == labels).sum().item() #ver cuantas acertaron, si acerto suma 1 si no 0
    # Medir % precision
    acc = 100.0 * n_correct/n_samples
    
    print(f'accuracy = {acc}')
        
        